## ------- SCC-ICMC-USP - 2025/2 -------

**SCC0275** - Introdução a Ciencia de Dados

**Profª Roseli A. F. Romero**  
**Monitores:**
- Thiago Rafael
- Paulo Henrique

#### Atividade 4 - 08/09/2025

**Formalização do Grupo (Nome e NUSP)**
- Integrante 1:
- Integrante 2:

---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Para esta lista de exercícios, considere a base de dados __Company Financial Dataset__, disponibilizada no [Kaggle](https://www.kaggle.com/datasets/atharvaarya25/financials/data). O arquivo do conjunto está disponível tanto no Tidia com o nome de __Financials.csv__.

In [2]:
# Celula de importacoes mais utilizadas (sinta-se livre para adicionar mais)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

---
### Item 1
O dataset Financials.csv possui colunas numéricas e categóricas. Entretanto, há registros inconsistentes, como valores não numéricos em colunas que deveriam ser numéricas.

- Carregue o dataset em um DataFrame do Pandas e identifique quais colunas deveriam ser numéricas, mas possuem registros inválidos (strings, caracteres especiais, etc.).  
- Mostre exemplos desses registros e explique brevemente como inconsistências desse tipo podem impactar modelos de Machine Learning.

In [36]:
# Carregar CSV (ajuste o caminho se precisar)
dados = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Financials.csv")

print(dados.head())
print()
print(dados.info())

      Segment  Country     Product   Discount Band   Units Sold   \
0  Government   Canada   Carretera            None    $1,618.50    
1  Government  Germany   Carretera            None    $1,321.00    
2   Midmarket   France   Carretera            None    $2,178.00    
3   Midmarket  Germany   Carretera            None      $888.00    
4   Midmarket   Mexico   Carretera            None    $2,470.00    

   Manufacturing Price   Sale Price   Gross Sales   Discounts         Sales   \
0                $3.00       $20.00    $32,370.00        $-      $32,370.00    
1                $3.00       $20.00    $26,420.00        $-      $26,420.00    
2                $3.00       $15.00    $32,670.00        $-      $32,670.00    
3                $3.00       $15.00    $13,320.00        $-      $13,320.00    
4                $3.00       $15.00    $37,050.00        $-      $37,050.00    

          COGS        Profit         Date  Month Number  Month Name   Year  
0   $16,185.00    $16,185.00   01

Colunas consideradas NUMÉRICAS são' Units Sold ', ' Manufacturing Price ', ' Sale Price ', ' Discounts ', ' Profit ', 'Date', 'Month Number', 'Year'.

 Colunas consideradas CATEGÓRICAS são 'Segment', 'Country', ' Product ', ' Discount Band ', ' Gross Sales ', ' Sales ', ' COGS ', ' Month Name '

Essas inconsistências afetam diretamente a qualidade dos dados porque modelos de Machine Learning não conseguem trabalhar com strings em colunas que deveriam ser numéricas. Dessa forma, quando isso acontece, o treinamento pode falhar ou obrigar o descarte da variável, perdendo a informação.

Além disso, quando números aparecem em formatos incorretos, como “$1,200” interpretado como texto em vez de valor numérico, a informação também é perdida.
Portanto, limpar e padronizar os dados é uma etapa crucial de pré-processamento, pois garante que as variáveis tenham o formato adequado e preserva a informação necessária para que o modelo aprenda de forma confiável.

### Item 2
- Calcule a porcentagem de valores faltantes em cada coluna.  
- Proponha uma estratégia de tratamento para pelo menos duas colunas com valores faltantes (ex.: remoção, imputação com média, mediana, moda, regressão etc.).  
- Justifique sua escolha e discuta por que diferentes estratégias podem ser mais adequadas para diferentes variáveis.  

In [38]:
# Substituir marcadores de vazio por NaN
dados.replace(["None", "-", " $-   "], np.nan, inplace=True)

# Calcular porcentagem de faltantes
faltantes = (dados.isna().sum() / len(dados)) * 100
print(faltantes[faltantes > 0])

Discounts     7.571429
Profit        0.714286
dtype: float64


No conjunto de dados analisado, foram identificadas apenas duas colunas com valores faltantes: Discount Band e Discounts.

A coluna Discount Band é de caráter categórico, indicando a faixa de desconto aplicada em determinada venda. Quando não há informação disponível, a imputação com a moda talvez é a opção mais adequada. Essa escolha preserva a coerência dos dados.

Já a coluna Discounts é uma variável numérica, que representa o valor de desconto aplicado. Nesse caso, o tratamento pode ser feito por meio da substituição dos valores ausentes por zero, pois a ausência de informação sobre desconto pode ser interpretada como a não possuir desconto naquela transação.

Diferentes estratégias são necessárias porque variáveis categóricas não podem ser tratadas da mesma forma que as numéricas. Além disso, tudo depende do contexto, ou seja, escolha deve sempre considerar tanto o tipo da variável quanto o contexto do negócio.

### Item 3
Considere que alguns atributos financeiros possuem escalas muito diferentes (por exemplo, receitas na casa dos milhões e outras variáveis em centenas).

- Aplique padronização e normalização em pelo menos duas colunas numéricas.  
- Compare os resultados e explique em quais cenários a padronização ou normalização seria mais indicada.  

In [30]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Strip whitespace from column names
dados.columns = dados.columns.str.strip()

# Limpar a coluna 'Sales' removendo '$' e ',' e converter para numérico
dados['Sales'] = dados['Sales'].astype(str).str.replace(r'[$,]', '', regex=True)
dados['Sales'] = pd.to_numeric(dados['Sales'], errors='coerce')

# Selecionar as colunas
num_cols = dados[["Sales", "Year"]]

# Padronização (Z-score: média = 0, desvio padrão = 1)
scaler_std = StandardScaler()
sales_year_std = scaler_std.fit_transform(num_cols)

# Normalização (Min-Max: valores entre 0 e 1)
scaler_minmax = MinMaxScaler()
sales_year_norm = scaler_minmax.fit_transform(num_cols)

# Criar DataFrame com os resultados
resultados = pd.DataFrame({
    "Sales_original": num_cols["Sales"],
    "Year_original": num_cols["Year"],
    "Sales_std": sales_year_std[:,0],
    "Year_std": sales_year_std[:,1],
    "Sales_norm": sales_year_norm[:,0],
    "Year_norm": sales_year_norm[:,1],
})

print(resultados.head())

   Sales_original  Year_original  Sales_std  Year_std  Sales_norm  Year_norm
0         32370.0           2014  -0.580152   0.57735    0.026535        1.0
1         26420.0           2014  -0.605304   0.57735    0.021394        1.0
2         32670.0           2014  -0.578884   0.57735    0.026794        1.0
3         13320.0           2014  -0.660682   0.57735    0.010077        1.0
4         37050.0           2014  -0.560368   0.57735    0.030578        1.0


Na transformação da coluna Sales, a padronização (Z-score) gerou valores próximos de zero, negativos para vendas abaixo da média e positivos para vendas acima da média. Já a normalização, reescalou todos os valores para o intervalo entre 0 e 1, mantendo apenas a proporção relativa entre o menor e o maior valor.

Na coluna Year, como todos os registros printados tinham o mesmo valor, a padronização resultou em um valor constante (0.5773) e a normalização em 1.0 para todos os casos.


A padronização é mais indicada quando queremos destacar o desvio em relação à média e quando os dados seguem uma distribuição aproximadamente normal, como em regressões. Já a normalização é mais adequada quando o objetivo é colocar variáveis em uma escala comum [0,1], especialmente em algoritmos baseados em distância (k-NN, redes neurais, SVM), para que nenhuma variável domine pelo simples fato de ter valores em escala maior.

### Item 4

O dataset contém colunas categóricas que precisarão ser transformadas em variáveis numéricas para uso em modelos de aprendizado de máquina.

- Selecione uma coluna categórica e aplique One-Hot Encoding e Label Encoding.
- Compare os dois métodos, discutindo as vantagens e desvantagens de cada abordagem.
- Dê exemplos de modelos de Machine Learning que podem se beneficiar de cada tipo de codificação.

In [31]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Exemplo com a coluna categórica
coluna = dados["Country"]

# Label Encoding
le = LabelEncoder()
country_label = le.fit_transform(coluna)

# One-Hot Encoding
country_onehot = pd.get_dummies(coluna, prefix="Country")

# Resultado
print("Label Encoding:", country_label[:5])
print("One-Hot Encoding:\n", country_onehot.head())


Label Encoding: [0 2 1 2 3]
One-Hot Encoding:
    Country_Canada  Country_France  Country_Germany  Country_Mexico  \
0            True           False            False           False   
1           False           False             True           False   
2           False            True            False           False   
3           False           False             True           False   
4           False           False            False            True   

   Country_United States of America  
0                             False  
1                             False  
2                             False  
3                             False  
4                             False  


O Label Encoding converte cada categoria em um número inteiro, sendo uma técnica simples e eficiente em termos de memória. Nesse sentido, a sua principal vantagem é a compactação dos dados, no entanto, cria uma ordem artificial entre categorias que pode induzir erros em modelos sensíveis a relações numéricas. Esse método é mais adequado para algoritmos baseados em árvores, como Decision Tree, Random Forest e XGBoost, que não interpretam os números como ordens, mas apenas como rótulos distintos.

Já o One-Hot Encoding transforma cada categoria em uma coluna binária, evitando a introdução de qualquer hierarquia entre elas. Desse modo, torna-se plausível afirmar que a vantagem é representar de forma mais fiel variáveis nominais, contudo a desvantagem é o aumento da dimensionalidade, já que é criada várias colunas. Esse tipo de codificação é mais indicado para modelos que dependem de distâncias ou pesos lineares, como Regressão Logística, KNN, SVM e Redes Neurais.

### Item 5

Considere o conjunto de colunas numéricas do conjunto.

- Aplique __PCA__ (Principal Component Analysis) para reduzir a dimensionalidade, mantendo pelo menos __90%__ da variância explicada e mostre o número de componentes necessários.
- Interprete os resultados: quais vantagens a redução de dimensionalidade pode trazer em problemas de ciência de dados? Cite também um possível risco ou desvantagem dessa técnica.

In [33]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Selecionar apenas colunas numéricas
num_cols = dados.select_dtypes(include=['float64','int64'])

# Imputar valores faltantes com a média (ou outra estratégia adequada)
imputer = SimpleImputer(strategy='mean')
num_imputed = imputer.fit_transform(num_cols)


# Padronizar os dados antes do PCA
scaler = StandardScaler()
num_scaled = scaler.fit_transform(num_imputed)

# Aplicar PCA sem definir o número de componentes inicialmente
pca = PCA()
pca.fit(num_scaled)

# Variância acumulada
explained_variance = pca.explained_variance_ratio_.cumsum()

# Quantos componentes para atingir 90%
n_components_90 = (explained_variance >= 0.90).argmax() + 1

print("Número de componentes para ≥90% da variância:", n_components_90)

Número de componentes para ≥90% da variância: 5


O PCA mostrou que, embora o dataset tenha várias colunas numéricas, apenas 5 componentes principais já são suficientes para reter 90% da variância explicada. Isso significa que quase toda a informação contida nos atributos originais pode ser representada em um espaço de dimensão muito menor.

Como vantagem, temos que a redução da dimensionalidade ajuda a diminuir a complexidade computacional, acelera o treinamento de modelos e pode reduzir redundâncias entre variáveis altamente correlacionadas. Além disso, ao projetar os dados em poucas dimensões, torna-se mais fácil visualizar padrões e tendências em gráficos.

Em contrapartida, a principal desvantagem é a perda de interpretabilidade, já que os novos componentes são combinações lineares de várias variáveis originais, dificultando a explicação dos resultados em termos de atributos do mundo real. Em alguns casos, também pode ocorrer uma pequena perda de informação relevante.